In [11]:
import math
import os
import time

from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives import serialization
from cryptography.hazmat.primitives.asymmetric import rsa


In [12]:
def generate_random_data(byte_count):
    """Генерує випадкові дані заданого розміру та вимірює тривалість генерації.

    Аргументи:
        byte_count (int): Кількість байтів для генерації.

    Повертає:
        tuple: Згенеровані дані у вигляді байтів та час, витрачений на їх створення.
    """
    start = time.time()  # Початок вимірювання часу
    random_bytes = os.urandom(byte_count)  # Створення випадкових байтів
    duration = time.time() - start  # Обчислення часу виконання

    # Виведення перших 64 символів у вигляді hex
    print(f'Згенеровані дані: {random_bytes.hex()[:64]}... (64 символи в hex)')

    return random_bytes, duration

In [13]:
def compute_shannon_entropy(byte_data):
    """Розраховує ентропію Шеннона для заданих байтових даних.

    Аргументи:
        byte_data (bytes): Масив байтів для аналізу.

    Повертає:
        float: Ентропія даних у бітах на байт.
    """
    # Ймовірності появи кожного байта
    probabilities = [byte_data.count(byte) / len(byte_data) for byte in set(byte_data)]
    # Формула Шеннона
    return -sum(p * math.log2(p) for p in probabilities)

In [14]:
def rsa_key_generation(bits=2048):
    """Створює пару RSA-ключів заданого розміру та вимірює час генерації.

    Аргументи:
        bits (int): Розмір ключа в бітах (типово 2048).

    Повертає:
        tuple: Приватний ключ RSA та час, витрачений на генерацію.
    """
    start = time.time()  # Початок вимірювання часу
    private_key = rsa.generate_private_key(public_exponent=65537, key_size=bits, backend=default_backend())
    duration = time.time() - start  # Час генерації

    print(f'RSA-ключ ({bits}-бітний) успішно створений')

    return private_key, duration

In [15]:
def evaluate_generation(byte_size, rsa_size):
    """Порівнює генерацію випадкових даних і ключів RSA за часом та ентропією.

    Аргументи:
        byte_size (int): Кількість байтів для генерації випадкових даних.
        rsa_size (int): Розмір RSA-ключа в бітах.
    """
    # Тест генерації випадкових байтів
    print(f"\n{'=' * 40}\nТест: Генерація випадкових даних ({byte_size} байтів):")
    random_data, gen_time = generate_random_data(byte_size)
    entropy_random = compute_shannon_entropy(random_data)
    print(f'• Час генерації: {gen_time:.6f} секунд')
    print(f'• Ентропія даних: {entropy_random:.6f} біт на байт')
    print('-' * 40)

    # Тест генерації RSA ключів
    print(f"\n{'=' * 40}\nТест: Генерація RSA-ключів ({rsa_size} біт):")
    rsa_key, rsa_time = rsa_key_generation(rsa_size)
    public_key_bytes = rsa_key.public_key().public_bytes(
        encoding=serialization.Encoding.PEM,
        format=serialization.PublicFormat.SubjectPublicKeyInfo
    )  # Публічний ключ у форматі PEM
    entropy_public_key = compute_shannon_entropy(public_key_bytes)

    print(f'• Час генерації: {rsa_time:.6f} секунд')
    print(f'• Довжина публічного ключа: {len(public_key_bytes)} байтів')
    print(f'• Ентропія публічного ключа: {entropy_public_key:.6f} біт на байт')
    print('-' * 40)

    # Підсумки
    print(f"\n{'=' * 40}\nРезультати порівняння:")
    print('Випадкові дані:')
    print(f'  - Час генерації: {gen_time:.6f} секунд')
    print(f'  - Ентропія: {entropy_random:.6f} біт на байт')
    print(f'\nRSA-ключі ({rsa_size} біт):')
    print(f'  - Час генерації: {rsa_time:.6f} секунд')
    print(f'  - Довжина ключа: {len(public_key_bytes)} байтів')
    print(f'  - Ентропія: {entropy_public_key:.6f} біт на байт')
    print('=' * 40)


# Виконання тесту
evaluate_generation(byte_size=1024, rsa_size=2048)



Тест: Генерація випадкових даних (1024 байтів):
Згенеровані дані: 3267276c1635b47c4e78954ca72b948907d12e5aad2891974ac5ca9e378cc7b6... (64 символи в hex)
• Час генерації: 0.000000 секунд
• Ентропія даних: 7.795693 біт на байт
----------------------------------------

Тест: Генерація RSA-ключів (2048 біт):
RSA-ключ (2048-бітний) успішно створений
• Час генерації: 0.185000 секунд
• Довжина публічного ключа: 451 байтів
• Ентропія публічного ключа: 5.849175 біт на байт
----------------------------------------

Результати порівняння:
Випадкові дані:
  - Час генерації: 0.000000 секунд
  - Ентропія: 7.795693 біт на байт

RSA-ключі (2048 біт):
  - Час генерації: 0.185000 секунд
  - Довжина ключа: 451 байтів
  - Ентропія: 5.849175 біт на байт
